# Uber:   Given locations of customers and drivers, make an "optimal" assignment.

In [1]:
from pprint import pprint

In [2]:
# Let's consider two customers, C1 and C2, we provide their locations:  [lat,lng]
C1_location = [33.780775, -84.386301]
C2_location = [33.778988, -84.363791]

# And two available drivers, D1 and D2
D1_location = [33.793711, -84.317408]
D2_location = [33.776812, -84.356153]

### We will use the `geopy` package to compute distances between two locations (it considers the curvature of the earth for us!)

In [3]:
from geopy.distance import geodesic

In [4]:
# We will store distances in a dictionary, with key 'CustomerName_DriverName'
distances = {}
distances['C1_D1'] = geodesic(C1_location,D1_location).miles
distances['C1_D2'] = geodesic(C1_location,D2_location).miles

distances['C2_D1'] = geodesic(C2_location,D1_location).miles
distances['C2_D2'] = geodesic(C2_location,D2_location).miles

pprint(distances)

{'C1_D1': 4.063663522199328,
 'C1_D2': 1.7564928121576306,
 'C2_D1': 2.8556454250592753,
 'C2_D2': 0.46447745846674}


### We are now ready to form a Linear Program to find an optimal assignment.  

### There are a number of packages to do optimization, we will use `PuLP` that is open source and free, but works much the same way as a commercial product

In [5]:
from pulp import *

### We use the PuLP function `LpProblem` to define a model instance, with an objective we will want to Minimize

In [6]:
model = LpProblem("Uber Assignment",LpMinimize)

### We will define a varible for each Customer/Driver pair.  The variable names will be of the form, `X_CustName_DriverName`,  so `X_C1_D2` for example.  To have PuLP do this, we first make a List of these pairs,`all_pairs`, without the `X_` prepended.

In [7]:
# Each arc variable is Binary 0,1
assignment_vars = LpVariable.dicts("X",['C1_D1','C1_D2','C2_D1','C2_D2'],cat='Binary')
pprint(assignment_vars)

{'C1_D1': X_C1_D1, 'C1_D2': X_C1_D2, 'C2_D1': X_C2_D1, 'C2_D2': X_C2_D2}


### We can now add our objective function to `model` which is to Minimize the sum of distance travelled

In [8]:
obj=''
for var in ['C1_D1','C1_D2','C2_D1','C2_D2']:
    obj += distances[var]*assignment_vars[var]
model += lpSum(obj), "Cost of each Customer Driver Assignment"
pprint(model)

Uber Assignment:
MINIMIZE
4.063663522199328*X_C1_D1 + 1.7564928121576306*X_C1_D2 + 2.8556454250592753*X_C2_D1 + 0.46447745846674*X_C2_D2 + 0.0
VARIABLES
0 <= X_C1_D1 <= 1 Integer
0 <= X_C1_D2 <= 1 Integer
0 <= X_C2_D1 <= 1 Integer
0 <= X_C2_D2 <= 1 Integer



### We now need our "node" constraints, each customer is assignned to exactly one driver, and each driver is assigned exactly one customer.  

### Now we add these constraints to our `model`

In [9]:
# Add constraint for each customer node
model += lpSum(assignment_vars['C1_D1'] + assignment_vars['C1_D2']) == 1, "C1"
model += lpSum(assignment_vars['C2_D1'] + assignment_vars['C2_D2']) == 1, "C2"
# Add constraint for each driver node
model += lpSum(assignment_vars['C1_D1'] + assignment_vars['C2_D1']) == 1, "D1"
model += lpSum(assignment_vars['C1_D2'] + assignment_vars['C2_D2']) == 1, "D2"

In [10]:
# Let's inspect our model
model

Uber Assignment:
MINIMIZE
4.063663522199328*X_C1_D1 + 1.7564928121576306*X_C1_D2 + 2.8556454250592753*X_C2_D1 + 0.46447745846674*X_C2_D2 + 0.0
SUBJECT TO
C1: X_C1_D1 + X_C1_D2 = 1

C2: X_C2_D1 + X_C2_D2 = 1

D1: X_C1_D1 + X_C2_D1 = 1

D2: X_C1_D2 + X_C2_D2 = 1

VARIABLES
0 <= X_C1_D1 <= 1 Integer
0 <= X_C1_D2 <= 1 Integer
0 <= X_C2_D1 <= 1 Integer
0 <= X_C2_D2 <= 1 Integer

In [11]:
# Let's solve the model and make sure it's status is good
model.solve()
print("Status:", LpStatus[model.status])

Status: Optimal


In [12]:
# Here is the solution
for v in model.variables():
    print(v.name, "=", v.varValue)

X_C1_D1 = 1.0
X_C1_D2 = 0.0
X_C2_D1 = 0.0
X_C2_D2 = 1.0


### Now that we have a solution, in the real world we need to disseminate that information, that would likely mean an update to our database to relect the assignment, and this will lead to a number of changes:
- Send this information to both the driver and customer
- Update live maps for admins and drivers and customer

### We will display the results on a Google Map

#### If you are using a local install, then you will need to install some packages, see https://buildmedia.readthedocs.org/media/pdf/jupyter-gmaps/latest/jupyter-gmaps.pdf for details.

#### You will also need a Google API key, which is also covered in this document

In [13]:
import gmaps
gmaps.configure(api_key='')

### Let's plot our customers and driver locations on a Google Map.  First, we collect a List of these locations.  And along the way we make a list of information that will be displayed when a Customer or Driver is clicked on.

In [14]:
customer_marker_locations=[C1_location, C2_location]
customer_info_boxes=['C1','C2']
pprint(customer_marker_locations)

driver_marker_locations=[D1_location,D2_location]
driver_info_boxes=['D1','D2']
pprint(driver_marker_locations)

[[33.780775, -84.386301], [33.778988, -84.363791]]
[[33.793711, -84.317408], [33.776812, -84.356153]]


In [15]:
fig=gmaps.figure()
customers_layer = gmaps.symbol_layer(
    customer_marker_locations, info_box_content = customer_info_boxes, fill_color='red', stroke_color='red', scale=6)
fig.add_layer(customers_layer)
drivers_layer = gmaps.symbol_layer(
    driver_marker_locations, info_box_content = driver_info_boxes, fill_color='blue', stroke_color='blue', scale=6)
fig.add_layer(drivers_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Now, let's draw a line between all the optimal assignments

In [16]:
# Loop through all the assignment variables
for v in model.variables():
    # If the assignment is made, then draw a line
    if (v.varValue==1.0):
        # A little fancy footwork to strip out the customer and driver name from the variable name
        var_name_split = v.name.split("_")
        print(f"Our variable name split into strings and put into a List: {var_name_split}")
        # We can now grab the customer and driver name
        cust_name = var_name_split[1]
        driver_name = var_name_split[2]
        # Now we create a line between the customer and driver
        assignment_line = gmaps.Line(
          start=eval(cust_name  + '_location'),
          end=eval(driver_name + '_location'),
          stroke_weight=3.0
        )
        # and we add the line to our map
        drawing = gmaps.drawing_layer(features=[assignment_line])
        fig.add_layer(drawing)
fig

Our variable name split into strings and put into a List: ['X', 'C1', 'D1']
Our variable name split into strings and put into a List: ['X', 'C2', 'D2']


Figure(layout=FigureLayout(height='420px'))

## Homework/Classroom work

- Can you reformulate the problem so that we minimize the worst case?  This would, it seems in this case, change the assignment we got originally.